### Data insights - expansion

### Comentários e conclusões:

- Gostaria de trazer elementos de Infográfico para o Dashboard
- Tipos de gráficos utilizados:scatterplot, barras, rosca, área, radar chart
- É necessário encontrar uma forma de comunicar os resultados do PCA

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.options.display.max_columns = 200
from sklearn.decomposition import PCA

In [ ]:
compfin = pd.read_pickle('compfin.pkl')

In [ ]:
compfin.head()

In [ ]:
print(compfin.columns.values)

### Let's expand all ideas to further converge them in a report

- barr
- scatter
- histogram
- explore some cool diagrams

In [ ]:
# restrinjo a analise aos projetos com dados
compfin = compfin[compfin.has_detailed_info]
compfin = compfin.assign(id_org = lambda x: x.organizacao + ": " + x.nome_projeto)
compfin=compfin.sort_values('valor_total', ascending = False)

In [ ]:
plt.figure(figsize=(20, 20))
plt.title("Valor total e solicitado iCSHG")
sns.set(style="whitegrid")
ax = sns.barplot(y= 'id_org', x='valor_total',
            data=compfin[['id_org', 'valor_total', 'solicitado_iCSHG']],
            orient="h", color = "b")
ax = sns.barplot(y= 'id_org', x='solicitado_iCSHG',
            data=compfin[['id_org', 'valor_total', 'solicitado_iCSHG']],
            orient="h")
plt.show()

- há projetos pequenos totalmente fundeados pelo iCSHG
- o % de custeio do iCHSG varia bastante mesmo em projetos de porte equivalente

## Ideia: um gráfico de área ficaria muito interessante (qudrados)

In [ ]:
n_largest_iCSHG = compfin.groupby(['id_org']).solicitado_iCSHG.sum().nlargest(5).to_frame()
n_largest_iCSHG.reset_index(inplace=True)

plt.figure(figsize=(20, 10))
plt.title("Valor solicitado iCSHG")
sns.set(style="whitegrid")
ax = sns.barplot(y= 'solicitado_iCSHG', x='id_org',
            data=n_largest_iCSHG,
            orient="v", palette = "rocket")
plt.show()

In [ ]:
sns.set(style="white")
sns.relplot(x="per_capita", y="beneficiarios", 
            size="solicitado_iCSHG", sizes=(100, 1000), 
            data=compfin,
            height = 8,
            hue = 'area',
            palette="muted")
plt.show()

- bolsas tem alto valor per capita e menor número de beneficiários
- duas classes de projeto: per_capita baixo x beneficiarios baixo

In [ ]:
sns.set(style="white")
sns.relplot(x="carga_horaria", y="per_capita", 
            size="beneficiarios", sizes=(60, 600), 
            data=compfin,
            height = 10,
            hue = 'area'
             )
plt.show()

- cutoff de carga horária
- maior numero de beneficiarios concentra em per capita baixo e carga horária baixa
- balanceado

#### Sensação de que os plots não passam a noção do patamar dos números totais e médios

In [ ]:
consolidated1 = compfin[compfin.valor_total > 0].groupby(['area']).\
nome_projeto.count().to_frame()
consolidated1.rename(columns={"nome_projeto": "counts"}, inplace=True)

consolidated2 = compfin[compfin.valor_total > 0].groupby(['area']).\
solicitado_iCSHG.sum().to_frame()

consolidated = pd.concat([consolidated1,consolidated2], axis=1, names = ['count','sum'])
consolidated.reset_index(inplace=True)
consolidated.sort_values('counts', inplace=True)

In [ ]:
sns.set(style="white", context="talk")

# Set up the matplotlib figure
f, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10, 10), sharex=True)

# Generate some sequential data
x = consolidated.area
y1 = consolidated.counts
sns.barplot(x=x, y=y1, palette="rocket", ax=ax1)
ax1.axhline(0, color="k", clip_on=False)
ax1.set_ylabel("counts")

# Center the data to make it diverging
x = consolidated.area
y2 = consolidated.solicitado_iCSHG
sns.barplot(x=x, y=y2, palette="rocket", ax=ax2)
ax2.axhline(0, color="k", clip_on=False)
ax2.set_ylabel("solicitado_iCSHG")

# Center the data to make it diverging
x = consolidated.area
y3 = consolidated.solicitado_iCSHG/consolidated.counts
sns.barplot(x=x, y=y3, palette="rocket", ax=ax3)
ax3.axhline(0, color="k", clip_on=False)
ax3.set_ylabel("ratio")

# Finalize the plot
sns.despine(bottom=True)
plt.setp(f.axes, yticks=[])
plt.tight_layout(h_pad=2)

- ordena
- a razao explode em Educação profissional

In [ ]:
print()
print("Informação financeira agregada (MM): nota-se que há alguns grandes players")

fin_agg = compfin[['id_org', 'ativo_2015','pl_2015',
                    'receitas_2015','despesas_2015']] # Odd! Too low! Have to multiply by 1000

In [ ]:
fin_agg = compfin[['id_org', 'ativo_2015',
                    'receitas_2015','despesas_2015']] # Odd! Too low! Have to multiply by 1000

sns.set(style="white")
sns.relplot(x="receitas_2015", y="despesas_2015", 
            size="ativo_2015", sizes=(60, 600), 

            data=fin_agg,
            height = 5             )
plt.show()

In [ ]:
sns.jointplot(x=fin_agg.receitas_2015, y=fin_agg.despesas_2015, data=fin_agg)

In [ ]:
#for id_org in fin_agg.id_org:
#    sns.barplot(data=fin_agg[fin_agg.id_org == id_org])
#    plt.title(id_org)
#    plt.show()

In [ ]:
print()
print("Receita detalhada (MM): nota-se que há alguns grandes players")

rec_det = compfin[compfin.has_detailed_info][['id_org', 'rec_empresas_2015', 'rec_fundacao_instituto_2015', 'rec_pf_2015', 'rec_governo_2015',
       'rec_outros_2015']]
for id_org in fin_agg.id_org:
    sns.barplot(data=rec_det[rec_det.id_org == id_org])
    plt.title(id_org)
    plt.show()

In [ ]:
print()
print("Despesa detalhada (MM): nota-se que há alguns grandes players")

des_det = compfin[compfin.has_detailed_info][['id_org', 'd_projetos', 'd_gerais_e_admin',
       'd_folha_de_pagamento', 'd_outros']]
for id_org in fin_agg.id_org:
    sns.barplot(data=des_det[des_det.id_org == id_org])
    plt.title(id_org)
    plt.show()

### Ideia: é necessário : clusterizar, achar visualização adequada (plotly tem radar chart), ou t-sne x  pca

### Ideia: É importante transmitir os totais. Vai enriquecer o leitor.

In [ ]:
rec_det = rec_det.set_index("id_org")

In [ ]:
rec_det=rec_det.fillna(0)

In [ ]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(rec_det)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['pc1', 'pc2'], index = rec_det.index)
sns.relplot(x = 'pc1', y = 'pc2',data = principalDf)
plt.show()

finalDf = pd.concat([principalDf, rec_det], axis = 1)
print(pca.explained_variance_ratio_)
print(pca.components_)
r1 = rec_det[rec_det.index.isin(finalDf.pc1.nlargest(2).index)]
sns.barplot(data =r1)
plt.show()
print(finalDf.pc1.nlargest(2))

r2 = rec_det[rec_det.index.isin(finalDf.pc2.nlargest(3).index)]
sns.barplot(data =r2)
plt.show()
print(finalDf.pc2.nlargest(3))

In [ ]:
des_det = des_det.set_index("id_org")

In [ ]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(des_det)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['pc1', 'pc2'], index = des_det.index)
sns.relplot(x = 'pc1', y = 'pc2',data = principalDf)
plt.show()

finalDf = pd.concat([principalDf, des_det], axis = 1)
print(pca.explained_variance_ratio_)
print(pca.components_)
r1 = des_det[des_det.index.isin(finalDf.pc1.nlargest(2).index)]
sns.barplot(data =r1)
plt.show()
print(finalDf.pc1.nlargest(2))

r2 = des_det[des_det.index.isin(finalDf.pc2.nlargest(3).index)]
sns.barplot(data =r2)
plt.show()
print(finalDf.pc2.nlargest(3))

 - vale a pena fazer % de despesa?
 - como colocar com fácil visualização